In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample.csv to sample.csv
Saving train.csv to train.csv


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('sample.csv')
train

,datetime,total
0,01.01.2005 00:00:00,NaN
1,01.01.2005 01:00:00,154139.8084
2,01.01.2005 02:00:00,157818.3593
3,01.01.2005 03:00:00,149310.6991
4,01.01.2005 04:00:00,138282.0380
...,...,...
35059,31.12.2008 19:00:00,249376.3608
35060,31.12.2008 20:00:00,246510.5725
35061,31.12.2008 21:00:00,226469.4133
35062,31.12.2008 22:00:00,199907.3942


In [4]:
train['total']=train['total'].fillna(method='ffill')
train['total']=train['total'].fillna(train['total'].mean())
train['total']=pd.to_numeric(train['total'])
train['total']=round(train['total'])
train

,datetime,total
0,01.01.2005 00:00:00,185144.0
1,01.01.2005 01:00:00,154140.0
2,01.01.2005 02:00:00,157818.0
3,01.01.2005 03:00:00,149311.0
4,01.01.2005 04:00:00,138282.0
...,...,...
35059,31.12.2008 19:00:00,249376.0
35060,31.12.2008 20:00:00,246511.0
35061,31.12.2008 21:00:00,226469.0
35062,31.12.2008 22:00:00,199907.0


In [5]:
import re 
#re.findall(r'[A-z]\d([A-Za-z]*)', 'A0V')
train['day'] = 0
train['month'] = 0
train['year'] = 0
train['time'] = 0
for index, row in train.iterrows():
  train.loc[index, 'day']= str(re.findall(r'^..', train.loc[index, 'datetime']))
  train.loc[index, 'month']=str(re.findall(r'[.][0-1][1-9][.]', train.loc[index, 'datetime']))
  train.loc[index, 'year']=str(re.findall(r'.... ', train.loc[index, 'datetime']))
  train.loc[index, 'time']=str(re.findall(r' ........', train.loc[index, 'datetime']))
train

,datetime,total,day,month,year,time
0,01.01.2005 00:00:00,185144.0,['01'],['.01.'],['2005 '],[' 00:00:00']
1,01.01.2005 01:00:00,154140.0,['01'],['.01.'],['2005 '],[' 01:00:00']
2,01.01.2005 02:00:00,157818.0,['01'],['.01.'],['2005 '],[' 02:00:00']
3,01.01.2005 03:00:00,149311.0,['01'],['.01.'],['2005 '],[' 03:00:00']
4,01.01.2005 04:00:00,138282.0,['01'],['.01.'],['2005 '],[' 04:00:00']
...,...,...,...,...,...,...
35059,31.12.2008 19:00:00,249376.0,['31'],['.12.'],['2008 '],[' 19:00:00']
35060,31.12.2008 20:00:00,246511.0,['31'],['.12.'],['2008 '],[' 20:00:00']
35061,31.12.2008 21:00:00,226469.0,['31'],['.12.'],['2008 '],[' 21:00:00']
35062,31.12.2008 22:00:00,199907.0,['31'],['.12.'],['2008 '],[' 22:00:00']


In [6]:
train = train.drop(['datetime'], axis=1)
train

,total,day,month,year,time
0,185144.0,['01'],['.01.'],['2005 '],[' 00:00:00']
1,154140.0,['01'],['.01.'],['2005 '],[' 01:00:00']
2,157818.0,['01'],['.01.'],['2005 '],[' 02:00:00']
3,149311.0,['01'],['.01.'],['2005 '],[' 03:00:00']
4,138282.0,['01'],['.01.'],['2005 '],[' 04:00:00']
...,...,...,...,...,...
35059,249376.0,['31'],['.12.'],['2008 '],[' 19:00:00']
35060,246511.0,['31'],['.12.'],['2008 '],[' 20:00:00']
35061,226469.0,['31'],['.12.'],['2008 '],[' 21:00:00']
35062,199907.0,['31'],['.12.'],['2008 '],[' 22:00:00']


In [7]:
from sklearn import preprocessing
 

l1 = preprocessing.LabelEncoder()
l1.fit(train['day'])
train['day'] = l1.transform(train['day'])

l2 = preprocessing.LabelEncoder()
l2.fit(train['month'])
train['month'] = l2.transform(train['month'])

l3 = preprocessing.LabelEncoder()
l3.fit(train['year'])
train['year'] = l3.transform(train['year'])

l4 = preprocessing.LabelEncoder()
l4.fit(train['time'])
train['time'] = l4.transform(train['time'])

train

,total,day,month,year,time
0,185144.0,0,0,0,0
1,154140.0,0,0,0,1
2,157818.0,0,0,0,2
3,149311.0,0,0,0,3
4,138282.0,0,0,0,4
...,...,...,...,...,...
35059,249376.0,30,10,3,19
35060,246511.0,30,10,3,20
35061,226469.0,30,10,3,21
35062,199907.0,30,10,3,22


In [8]:
X = train.drop(['total'], axis = 1)
y = train['total']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

X_st

array([[-1.67380491, -1.60078063, -1.34219223, -1.66132477],
       [-1.67380491, -1.60078063, -1.34219223, -1.51686175],
       [-1.67380491, -1.60078063, -1.34219223, -1.37239873],
       ...,
       [ 1.73525035,  1.29620995,  1.34035613,  1.37239873],
       [ 1.73525035,  1.29620995,  1.34035613,  1.51686175],
       [ 1.73525035,  1.29620995,  1.34035613,  1.66132477]])

In [9]:
X = train[['day', 'month', 'year', 'time']].values

y = train['total'].values
features = pd.Series(['day', 'month', 'year', 'time'])
features

0      day
1    month
2     year
3     time
dtype: object

In [10]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
# Применим линейную регрессию к нашему набору данных 
slr.fit(X, y)
# так как у нас много признаков, а не один как в предыдушем примере,
# то у нас будет много разных коэффицентов для каждого признака
# сохраним их все в датафрейм
coeff_df = pd.DataFrame(slr.coef_, columns=['Coefficient'])  
coeff_df['features'] = features

coeff_df

,Coefficient,features
0,84.836221,day
1,-758.721302,month
2,-8055.230927,year
3,4433.629308,time


In [11]:
X = train[['time']].values
y = train['total'].values
slr = LinearRegression()
slr.fit(X, y)
y_pred = slr.predict(X)

print('Slope: ', slr.coef_)
print('Intercept:', slr.intercept_)

Slope:  [4433.6293075]
Intercept: 134157.71699064568


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('MSE: {:.3f}'.format(mean_squared_error(y, y_pred)))

MSE: 1130605527.230


In [13]:















test = pd.read_csv('sample.csv')

In [14]:
import re 
#re.findall(r'[A-z]\d([A-Za-z]*)', 'A0V')
test['day'] = 0
test['month'] = 0
test['year'] = 0
test['time'] = 0
for index, row in test.iterrows():
  test.loc[index, 'day']= str(re.findall(r'^..', test.loc[index, 'datetime']))
  test.loc[index, 'month']=str(re.findall(r'[.][0-1][1-9][.]', test.loc[index, 'datetime']))
  test.loc[index, 'year']=str(re.findall(r'.... ', test.loc[index, 'datetime']))
  test.loc[index, 'time']=str(re.findall(r' ........', test.loc[index, 'datetime']))
test

,datetime,total,day,month,year,time
0,01.01.2009 00:00:00,0,['01'],['.01.'],['2009 '],[' 00:00:00']
1,01.01.2009 01:00:00,0,['01'],['.01.'],['2009 '],[' 01:00:00']
2,01.01.2009 02:00:00,0,['01'],['.01.'],['2009 '],[' 02:00:00']
3,01.01.2009 03:00:00,0,['01'],['.01.'],['2009 '],[' 03:00:00']
4,01.01.2009 04:00:00,0,['01'],['.01.'],['2009 '],[' 04:00:00']
...,...,...,...,...,...,...
4339,30.06.2009 19:00:00,0,['30'],['.06.'],['2009 '],[' 19:00:00']
4340,30.06.2009 20:00:00,0,['30'],['.06.'],['2009 '],[' 20:00:00']
4341,30.06.2009 21:00:00,0,['30'],['.06.'],['2009 '],[' 21:00:00']
4342,30.06.2009 22:00:00,0,['30'],['.06.'],['2009 '],[' 22:00:00']


In [15]:
from sklearn import preprocessing
 

l1 = preprocessing.LabelEncoder()
l1.fit(test['day'])
test['day'] = l1.transform(test['day'])

l2 = preprocessing.LabelEncoder()
l2.fit(test['month'])
test['month'] = l2.transform(test['month'])

l3 = preprocessing.LabelEncoder()
l3.fit(test['year'])
test['year'] = l3.transform(test['year'])

l4 = preprocessing.LabelEncoder()
l4.fit(test['time'])
test['time'] = l4.transform(test['time'])

test

,datetime,total,day,month,year,time
0,01.01.2009 00:00:00,0,0,0,0,0
1,01.01.2009 01:00:00,0,0,0,0,1
2,01.01.2009 02:00:00,0,0,0,0,2
3,01.01.2009 03:00:00,0,0,0,0,3
4,01.01.2009 04:00:00,0,0,0,0,4
...,...,...,...,...,...,...
4339,30.06.2009 19:00:00,0,29,5,0,19
4340,30.06.2009 20:00:00,0,29,5,0,20
4341,30.06.2009 21:00:00,0,29,5,0,21
4342,30.06.2009 22:00:00,0,29,5,0,22


In [16]:
X = test.drop(['total','datetime'], axis = 1)
y = train['total']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

X_st

array([[-1.67162811, -1.46686867,  0.        , -1.66132477],
       [-1.67162811, -1.46686867,  0.        , -1.51686175],
       [-1.67162811, -1.46686867,  0.        , -1.37239873],
       ...,
       [ 1.64822659,  1.4571757 ,  0.        ,  1.37239873],
       [ 1.64822659,  1.4571757 ,  0.        ,  1.51686175],
       [ 1.64822659,  1.4571757 ,  0.        ,  1.66132477]])

In [22]:
X = test[['day', 'month', 'year', 'time']].values

y = train['total'].values
features = pd.Series(['day', 'month', 'year', 'time'])
features

0      day
1    month
2     year
3     time
dtype: object

In [ ]:
y_pred = slr.predict(X)
test['total']=y_pred
test

In [24]:
sample=test.drop(['day', 'month','year','time'],axis=1)
sample

,datetime,total
0,01.01.2009 00:00:00,0
1,01.01.2009 01:00:00,0
2,01.01.2009 02:00:00,0
3,01.01.2009 03:00:00,0
4,01.01.2009 04:00:00,0
...,...,...
4339,30.06.2009 19:00:00,0
4340,30.06.2009 20:00:00,0
4341,30.06.2009 21:00:00,0
4342,30.06.2009 22:00:00,0


In [25]:
sample.to_csv('Result_final.csv',index=False)
files.download("Result_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>